In [1]:
import rospy
import numpy as np
from panda_robot_custom import PandaRobotCustom
from tf.transformations import quaternion_from_euler, euler_from_quaternion
import my_log
import my_tools

log = my_log.logger()

rospy.init_node("franka_test", anonymous=True)

robot = PandaRobotCustom(log_level=20)
rospy.sleep(1) 

2023-09-13 18:52:59 [INFO] Initializing PandaRobot class
2023-09-13 18:53:00 [INFO] Loaded controlers: ['franka_state_controller', 'cartesian_impedance_example_controller']
2023-09-13 18:53:01 [INFO] Stiffness: {'translational_stiffness': 400.0, 'rotational_stiffness': 20.0, 'nullspace_stiffness': 0.5}
2023-09-13 18:53:06 [INFO] Response switch controllers: ok: True
2023-09-13 18:53:06 [INFO] Controllers: [name: "franka_state_controller"
state: "stopped"
type: "franka_control/FrankaStateController"
claimed_resources: 
  - 
    hardware_interface: "franka_hw::FrankaStateInterface"
    resources: [], name: "cartesian_impedance_example_controller"
state: "stopped"
type: "franka_example_controllers/CartesianImpedanceExampleController"
claimed_resources: 
  - 
    hardware_interface: "franka_hw::FrankaModelInterface"
    resources: []
  - 
    hardware_interface: "hardware_interface::EffortJointInterface"
    resources: 
      - panda_joint1
      - panda_joint2
      - panda_joint3
      -

In [2]:
trans = [0.3, 0.0, 0.6]
rot = [np.pi, 0.0, 0.0]
robot.cart_move(trans, quaternion_from_euler(rot[0], rot[1], rot[2]))

In [3]:
log.info(robot.get_cart_pose(frame1="panda_EE"))

2023-09-13 18:53:36 [ERROR] Transform could not be fetched ('panda_link0' to 'qbhand_flange1')
2023-09-13 18:53:36 [INFO] None
